In [1]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import json
import csv

In [ ]:
IMDB_URL_MOVIES = r'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
LETTERBOXD_MOVIES = r'https://letterboxd.com/dave/list/official-top-250-narrative-feature-films/'
ROTTEN_TOMATOES_MOVIES = r'https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/'
str1 = r'https://www.imdb.com'
str2 = r'https://letterboxd.com'

In [10]:
links1 = set()
links3 = set()
links4 = set()

data_imdb_movies = { 'Title' : [] , 'Link' : [] , 'Year' : [] , 'Rating_Classification' : [] , 'Runtime' : [] , 'Total_Ratings' : [] , 'Average_Rating' : [] , 'Genre' : [] , 'Director' : [] }
data_letterbox_movies = { 'Title' : [] , 'Link' : [] , 'Year' : []  , 'Runtime' : [] , 'Average_n_total_Rating' : [] , 'Genre' : [] , 'Director' : [] }
data_rotten_tomatoes_movies = { 'Title' : [] , 'Link' : [] , 'Year' : [] , 'Runtime' : [] , 'Rating_Classification' : [] , 'Runtime' : [] , 'Total_Ratings' : [] , 'Critic_Score' : [] , 'Audience_Score' : [] , 'Genre' : [] , 'Director' : [] }

In [ ]:
 # IMDB Movies
try:
     # opening the page 
    driver = webdriver.Chrome()
    driver.get( IMDB_URL_MOVIES )
    driver.maximize_window()
    driver.execute_script("document.body.style.zoom='100%'")

    while True:
        # Create an explicit wait object
        wait = WebDriverWait(driver, 20)

         # Wait for an element to be present
        elements = wait.until(EC.presence_of_all_elements_located(( By.CSS_SELECTOR, 'a.ipc-title-link-wrapper[href]') ))

        html1 = driver.page_source

         # Creating a soup object 
        main_soup = BeautifulSoup( html1 , 'lxml' )

         # Getting the link
        link_attr = main_soup.select('a.ipc-title-link-wrapper[href]')

        for element in link_attr:
             
            link = element.get( 'href' )
            
             # Adding the imdb.com
            link = str1 + link

            if link in links1:
                continue
            else:
                links1.add(link)
                data_imdb_movies['Link'].append(link) 

                driver.get(link)
                sleep(1)

                 # Getting the data using bs4
                div_elem = wait.until(EC.presence_of_element_located(( By.CSS_SELECTOR, 'div.sc-70a366cc-0.bxYZmb') ))


                if div_elem:
                     # Getting the html 
                    html = driver.page_source

                     # Beautiful Soup object
                    soup_2 = BeautifulSoup( html , 'lxml' )

                    div_attr = soup_2.select( 'div.sc-70a366cc-0.bxYZmb')
                    
                     # Getting the data in div elements
                    for elements in div_attr:
                        
                         # Getting the Title 
                        title_attr = elements.select('span.hero__primary-text')

                        if  title_attr:
                   
                            title = title_attr[0].get_text().strip()

                            data_imdb_movies['Title'].append(title)
                        

                        else:

                            data_imdb_movies['Title'].append('')

                         # Getting the year , Rating classification and runtime
                        li_attr = elements.select('li.ipc-inline-list__item')
            
                        if li_attr:

                            if len(li_attr) == 3 :

                                year = li_attr[0].get_text()
                                data_imdb_movies['Year'].append(year)

                                rating_classification = li_attr[1].get_text()
                                data_imdb_movies['Rating_Classification'].append(rating_classification)
                
                                runtime = li_attr[2].get_text()
                                data_imdb_movies['Runtime'].append(runtime)

                            else:
                                
                                data_imdb_movies['Rating_Classification'].append('CHECK')

                                data = ''

                                for i in li_attr:

                                    x = i.get_text()
                                    
                                    if data:

                                        data += '|'

                                    data += x

                                data_imdb_movies['Year'].append(data)

                                data_imdb_movies['Runtime'].append('CHECK')


                                
                        else:

                            data_imdb_movies['year'].append('')
                            data_imdb_movies['Runtime'].append('')
                            data_imdb_movies['Rating_Classification'].append('')



                     # Getting the Total Ratings 
                    ratings_attr = soup_2.select('div.sc-d541859f-3.dwhNqC')

                    if ratings_attr:

                        total_Ratings = ratings_attr[0].get_text().strip()
                        data_imdb_movies['Total_Ratings'].append(total_Ratings)

                    else:

                        data_imdb_movies['Total_Ratings'].append('')

                     # Getting Average Rating
                    avg_rating_attr = soup_2.select('div[data-testid="hero-rating-bar__aggregate-rating__score"]')

                    if avg_rating_attr:

                        avg_rating = avg_rating_attr[0].get_text().strip()
                        data_imdb_movies['Average_Rating'].append(avg_rating)

                    else:

                        data_imdb_movies['Average_Rating'].append('')

                     # Getting Genre
                    genre_attr = soup_2.select('span.ipc-chip__text')

                    if genre_attr: 

                        if len(genre_attr) >= 3:

                            genre = genre_attr[0].get_text().strip() + '|' + genre_attr[1].get_text().strip() + '|' + genre_attr[2].get_text().strip()
                            data_imdb_movies['Genre'].append(genre)

                        else:

                            genre = ''

                            for i in genre_attr:
                                 
                                x = i.get_text()

                                if genre:

                                    genre += '|'

                                genre += x

                            data_imdb_movies['Genre'].append(genre)
                                

                    else:

                        data_imdb_movies['Genre'].append('')

                     # Getting the Director
                    dir_attr = soup_2.select('a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')

                    if dir_attr:

                        director = dir_attr[0].get_text().strip()
                        data_imdb_movies['Director'].append(director)

                    else:

                        data_imdb_movies['Director'].append('')

                driver.back()
                sleep(1)

                 # Breaking 
                if len(links1)>= 250:
                    break
                
                else:
                    continue


except WebDriverException as web_err:
    print(f'Selenium WebDriver error occurred: {web_err}')
except Exception as e:
    print(f'An error occurred: {e}')

finally:
    driver.quit()

In [23]:
 # Saving the IMDB Movies as a json file

with open('IMDB_DATA.json', mode='w', encoding='utf-8') as file:
    # Write the data dictionary to the file as JSON
    json.dump(data_imdb_movies, file, ensure_ascii=False, indent=4)


In [7]:
 # scrolling in small increments
increment = 0.99 # Scroll by 50% of the remaining page height

In [15]:
 # LETTERBOXD MOVIES
try:
     # opening the page 
    driver = webdriver.Chrome()
    driver.get( LETTERBOXD_MOVIES )
    driver.maximize_window()
    driver.execute_script("document.body.style.zoom='100%'")

    while True:
        # Create an explicit wait object
        wait = WebDriverWait(driver, 20)

         # Wait for an element to be present
        elements = wait.until(EC.presence_of_all_elements_located(( By.CSS_SELECTOR, 'a.frame[href]') ))

        for i in range(4):

            scroll_position = driver.execute_script("return window.pageYOffset + window.innerHeight + window.innerHeight * {0}".format(increment))
    
             # Scroll by the incremented amount
            driver.execute_script(f"window.scrollTo(0, {scroll_position});")

            sleep(3)


        html1 = driver.page_source

         # Creating a soup object 
        main_soup = BeautifulSoup( html1 , 'lxml' )

        link_attr = main_soup.select('a.frame')

        print(len(link_attr))
        
        for element in link_attr:

            link = element.get('href')
            
             # Adding letterboxd.com
            link = str2 + link 
            
            if link in links3:
                continue
            else:
                #links3.add(link)
                #data_letterbox_movies['Link'].append(link) 

                driver.get(link)
                sleep(1)

                title_elem = wait.until(EC.presence_of_all_elements_located(( By.CSS_SELECTOR, 'span.name.js-widont.prettify') ))

                if title_elem:
                    
                    html = driver.page_source
                    
                     # Beautiful Soup object
                    soup_2 = BeautifulSoup( html , 'lxml' )

                     # getting the Title
                    title_attr = soup_2.select('span.name.js-widont.prettify')

                    if title_attr :

                        title = title_attr[0].get_text()
                        data_letterbox_movies['Title'].append(title)

                    else:

                        data_letterbox_movies['Title'].append('')

                     # Getting the year
                    year_attr = soup_2.select('div.releaseyear')

                    if year_attr :

                        year = year_attr[0].get_text()
                        data_letterbox_movies['Year'].append(year)

                    else:

                        data_letterbox_movies['Year'].append('')

                     # Getting the Director
                    dir_attr = soup_2.select('span.directorlist')

                    if dir_attr :

                        director = dir_attr[0].get_text()
                        data_letterbox_movies['Director'].append(director)

                    else:

                        data_letterbox_movies['Director'].append('')    

                     # Getting the Runtime
                    runtime_attr = soup_2.select('p.text-link.text-footer')

                    if runtime_attr :

                        runtime = runtime_attr[0].get_text()
                        data_letterbox_movies['Runtime'].append(runtime)

                    else:

                        data_letterbox_movies['Runtime'].append('')                  
               
                     # Getting the average weighted rating and total ratings
                    a_tag = soup_2.select_one('span.average-rating a.tooltip.display-rating.-highlight')

                    
                    if a_tag:

                        original_title = a_tag.get('data-original-title')

                        data_letterbox_movies['Average_n_total_Rating'].append(original_title)
                    
                    else:

                        data_letterbox_movies['Average_n_total_Rating'].append('')

                    div_genre = soup_2.select('div[id="tab-genres"]')

                    for elements in div_genre:

                        genre_attr = elements.select('a.text-slug')

                        if genre_attr :
                            
                            genre = genre_attr[0].get_text()
                            data_letterbox_movies['Genre'].append(genre)

                        else :

                            data_letterbox_movies['Genre'].append('')
                
                links3.add(link)
                data_letterbox_movies['Link'].append(link)
                
                driver.back()
                sleep(1)

        next_button = main_soup.select('a.next')

        if next_button:
            
            for elements in  next_button:

                next_link = next_button[0].get('href')

                link = str2 + next_link

                driver.get(link) 
                
                continue 

        else:
            print('No Next Button')
            break     
                    




except WebDriverException as web_err:
    print(f'Selenium WebDriver error occurred: {web_err}')
except Exception as e:
    print(f'An error occurred: {e}')

finally:
    driver.quit()

100
100
50
No Next Button


In [16]:
 # Saving the Data 

with open('Letterbox_Data.json', mode='w', encoding='utf-8') as file:
    # Write the data dictionary to the file as JSON
    json.dump(data_letterbox_movies, file, ensure_ascii=False, indent=4)   

In [ ]:
 # ROTTEN TOMATOES MOVIES

try:
     # opening the page 
    driver = webdriver.Chrome()
    driver.get( ROTTEN_TOMATOES_MOVIES )
    driver.maximize_window()
    driver.execute_script("document.body.style.zoom='100%'")

    while True:
        # Create an explicit wait object
        wait = WebDriverWait(driver, 20)

         # Wait for an element to be present
        elements = wait.until(EC.presence_of_all_elements_located(( By.CSS_SELECTOR, 'a.title[href]') ))

        html1 = driver.page_source

         # Creating a soup object 
        main_soup = BeautifulSoup( html1 , 'lxml' )

        link_attr = main_soup.select('a.title')

        for element in link_attr:
            
            link = element.get('href')

            if link in links4:
                continue

            else:

                links4.add(link)
                data_rotten_tomatoes_movies['Link'].append(link)

                driver.get(link)
                sleep(1)

                title_elem = wait.until(EC.presence_of_all_elements_located(( By.CSS_SELECTOR, 'rt-text[slot="title"]') ))

                if title_elem:

                    html = driver.page_source
                    
                     # Beautiful Soup object
                    soup_2 = BeautifulSoup( html , 'lxml' )

                     # getting the Title
                    title_attr = soup_2.select('rt-text[slot="title"]')

                    if title_attr :

                        title = title_attr[0].get_text()
                        data_rotten_tomatoes_movies['Title'].append(title)

                    else:

                        data_rotten_tomatoes_movies['Title'].append('')
                    
                     # getting the rating ,release date and runtime
                    element1 = soup_2.select('rt-text[slot="metadataProp"]')

                    if element1 :

                        if len(element1) == 3:

                            rating = element1[0].get_text()
                            data_rotten_tomatoes_movies['Rating_Classification'].append(rating)

                            release_date = element1[1].get_text()
                            data_rotten_tomatoes_movies['Year'].append(release_date)

                            runtime = element1[2].get_text()
                            data_rotten_tomatoes_movies['Runtime'].append(runtime)

                        else :

                            data_rotten_tomatoes_movies['Rating_Classification'].append('CHECK')
                            
                            data = ''

                            for i in element1:

                                x = i.get_text()

                                if data:
                                    data += '|'

                                data += x

                            data_rotten_tomatoes_movies['Year'].append(data)

                            data_rotten_tomatoes_movies['Runtime'].append('CHECK')
                            
                    else:

                        data_rotten_tomatoes_movies['Rating_Classification'].append('')
                        data_rotten_tomatoes_movies['Year'].append('')
                        data_rotten_tomatoes_movies['Runtime'].append('')

                     # Getting Genre
                    genre_attr = soup_2.select('rt-text[slot="metadataGenre"]')

                    if genre_attr:

                        genre = genre_attr[0].get_text()
                        data_rotten_tomatoes_movies['Genre'].append(genre)

                    else:

                        data_rotten_tomatoes_movies['Genre'].append('')

                     # Getting the Critic score
                    critic_attr = soup_2.select('rt-text[slot="criticsScore"]')

                    if critic_attr:

                        critic_score = critic_attr[0].get_text()
                        data_rotten_tomatoes_movies['Critic_Score'].append(critic_score)

                    else:

                        data_rotten_tomatoes_movies['Critic_Score'].append('')

                     # Getting the audience score
                    audience_attr = soup_2.select('rt-text[slot="audienceScore"]')

                    if audience_attr:

                        audience_score = audience_attr[0].get_text()
                        data_rotten_tomatoes_movies['Audience_Score'].append(audience_score)

                    else:

                        data_rotten_tomatoes_movies['Audience_Score'].append('')


                     # Getting the Total Ratings
                    ratings_attr = soup_2.select('rt-link[slot="audienceReviews"]')

                    if ratings_attr:

                        total_Ratings = ratings_attr[0].get_text()
                        data_rotten_tomatoes_movies['Total_Ratings'].append(total_Ratings)

                    else:

                        data_rotten_tomatoes_movies['Total_Ratings'].append('')

                     # Getting the Director
                    dir_attr = soup_2.select('p.name')

                    if dir_attr:

                        director = dir_attr[0].get_text()
                        data_rotten_tomatoes_movies['Director'].append(director)

                    else:

                        data_rotten_tomatoes_movies['Director'].append('')
                
                driver.back()
                sleep(1)

                # Breaking the loop at 200
                if len(links4) >= 299:
                    
                    break

                else :
                    continue



except WebDriverException as web_err:
    print(f'Selenium WebDriver error occurred: {web_err}')
except Exception as e:
    print(f'An error occurred: {e}')

finally:
    driver.quit()

In [9]:
 # Saving the Data
with open('Rotten_Tomatoes_Data.json', mode='w', encoding='utf-8') as file:
    # Write the data dictionary to the file as JSON
    json.dump(data_rotten_tomatoes_movies, file, ensure_ascii=False, indent=4)